In [11]:
from SwissArmyKnife import ClickHouseClient
import pandas as pd
from tqdm import tqdm
from StringIO import StringIO

In [6]:
ch_settings_df = pd.read_excel('/Users/artkaz/Documents/yacloud/settings/ch_cloud_analytics_connection.xlsx') #sys.argv[1]

ch_settings = {ch_settings_df.iloc[i,:]['field']: ch_settings_df.iloc[i,:]['value'] for i in ch_settings_df.index}


ch_client = ClickHouseClient(user = ch_settings['user'],
                             passw = ch_settings['password'],
                             verify = ch_settings['verify'],
                             host = ch_settings['host']
                             )

In [31]:
ch_tables = pd.read_csv(StringIO(ch_client.exec_sql("Select database, name FROM system.tables")), 
                        sep ='\t', names=['database', 'table'])

In [32]:
ch_tables = ch_tables[ch_tables['database'] == 'cloud_analytics']

In [33]:
ch_tables['temp_table'] = ch_tables['table'].apply(lambda s: s.find('_15')>0)

In [34]:
for table in tqdm(ch_tables[ch_tables['temp_table']]['table']):
    try:
        ch_client.exec_sql('DROP TABLE cloud_analytics.%s' %table)
    except:
        print 'err'


100%|██████████| 46/46 [00:18<00:00,  2.54it/s]
